 # Lab 6: 피드백 기반 재번역



 Lab 3-5 평가 결과와 US/GB 지역 타겟팅을 활용하여 레퍼런스에 가까운 번역을 생성합니다.



 ## 학습 목표



 - 레퍼런스 데이터에서 용어집/스타일 가이드 자동 추출

 - 평가 피드백을 활용한 번역 품질 개선

 - 재번역 후 충실도/용어/톤 재평가로 개선 확인



 ## 워크샵 전체 흐름



 ```

 Lab 1: 환경 설정 ✅

 ↓

 Lab 2: 번역 기법 비교 ✅

 ↓

 Lab 3: 품질 평가 - 충실도 ✅

 ↓

 Lab 4: 품질 평가 - 용어 일관성 ✅

 ↓

 Lab 5: 품질 평가 - 문화/톤 ✅

 ↓

 Lab 6: 피드백 기반 재번역 (현재)

 ↓

 Lab 7: 종합 분석

 ```

 ---

 ## 6.1 환경 설정 및 데이터 로드

In [1]:
import json
import os
from datetime import datetime

import boto3
import pandas as pd


In [2]:
REGION = "us-west-2"
OUTPUT_DIR = "lab_outputs"
DATA_DIR = "data"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION
)

CLAUDE_MODELS = {
    "sonnet": {
        "id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name": "Claude Sonnet 4",
    },
    "haiku": {
        "id": "us.anthropic.claude-haiku-4-20250514-v1:0",
        "name": "Claude Haiku 4",
    }
}

DEFAULT_MODEL = "sonnet"
MODEL_ID = CLAUDE_MODELS[DEFAULT_MODEL]["id"]

print(f"Bedrock 클라이언트 초기화 완료")
print(f"사용 모델: {CLAUDE_MODELS[DEFAULT_MODEL]['name']}")


Bedrock 클라이언트 초기화 완료
사용 모델: Claude Sonnet 4


In [3]:
def load_results(filename: str) -> dict:
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)

def load_json(filename: str) -> dict:
    filepath = os.path.join(DATA_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)

def save_results(data: dict, filename: str) -> str:
    filepath = os.path.join(OUTPUT_DIR, filename)
    data["_metadata"] = {
        "saved_at": datetime.now().isoformat(),
        "model": CLAUDE_MODELS[DEFAULT_MODEL]["name"]
    }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✓ 저장 완료: {filepath}")
    return filepath

def call_claude(prompt: str, model_id: str = MODEL_ID, max_tokens: int = 2048) -> str:
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType="application/json",
        accept="application/json"
    )
    response_body = json.loads(response["body"].read())
    return response_body["content"][0]["text"]


In [4]:
# 데이터 로드
workshop_config = load_results("workshop_config.json")
faithfulness_data = load_results("eval_faithfulness.json")
terminology_data = load_results("eval_terminology.json")
culture_tone_data = load_results("eval_culture_tone.json")

ko_faq = load_json("ko_faq.json")
en_us_faq = load_json("en-rUS_faq.json")
en_gb_faq = load_json("en-rGB_faq.json")

faq_items = workshop_config["faq_items"]
print(f"워크샵 FAQ 항목: {len(faq_items)}개")


워크샵 FAQ 항목: 6개


 ---

 ## 6.2 레퍼런스에서 용어집 자동 추출

In [5]:
GLOSSARY_EXTRACTION_PROMPT = """한국어-영어 번역 쌍을 분석하여 용어집을 추출해주세요.

## 번역 쌍
{translation_pairs}

## 추출 대상
1. 브랜드명 (번역 금지): Samsung Cloud, Galaxy 등
2. 기술 용어: 동기화, 백업, 복원 등
3. UI 용어: 설정, 휴지통 등

반드시 JSON 형식으로 응답:
```json
{{
  "brand_terms": {{"한국어": "영어"}},
  "technical_terms": {{"한국어": "영어"}},
  "ui_terms": {{"한국어": "영어"}}
}}
```"""


In [6]:
def extract_glossary(ko_data: dict, en_data: dict, locale: str) -> dict:
    pairs = []
    for key in list(ko_data.keys())[:20]:
        if key in en_data and ko_data[key] and en_data[key]:
            pairs.append(f"한국어: {ko_data[key]}\n영어: {en_data[key]}")
    
    prompt = GLOSSARY_EXTRACTION_PROMPT.format(translation_pairs="\n\n".join(pairs))
    response = call_claude(prompt)
    
    try:
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        else:
            json_str = response.strip()

        data = json.loads(json_str)
        combined = {}
        for category in data.values():
            if isinstance(category, dict):
                combined.update(category)
        return {"locale": locale, "glossary": combined, "categories": data}
    except:
        return {"locale": locale, "glossary": {}, "categories": {}}


In [7]:
print("US 영어 용어집 추출 중...")
us_glossary_data = extract_glossary(ko_faq, en_us_faq, "US")

print("US 영어 용어집:")
for ko, en in us_glossary_data["glossary"].items():
 print(f"  {ko} → {en}")

save_results(us_glossary_data, "glossary_us.json")


US 영어 용어집 추출 중...
US 영어 용어집:
  삼성 클라우드 → Samsung Cloud
  삼성 계정 → Samsung account
  삼성 인터넷 → Samsung Internet
  Samsung Notes → Samsung Notes
  Samsung Pass → Samsung Pass
  Samsung Daily → Samsung Daily
  동기화 → synchronization/sync
  백업 → backup
  복원 → restore
  백업/복원 → backup/restore
  연동 → linked
  서버 → server
  데이터 → data
  OS → OS
  RAM → RAM
  S/W → software
  설정 → Settings
  일정 → Events
  캘린더 → Calendar
  연락처 → Contacts
  갤러리 → Gallery
  리마인더 → Reminders
  AR 이모지 → AR emojis
  통화 기록 → Call logs
  시계 → Clock settings
  메시지 → Messages
  홈 화면 → Home screen layout
  애플리케이션 → Apps
  단말의 설정 → Device settings
  음성 녹음 → Voice recordings
  더 보기 → More
  앱스 화면 → Apps screen
  Contact us → Contact us
  1대1 문의하기 → 1:1 inquiry
✓ 저장 완료: lab_outputs/glossary_us.json


'lab_outputs/glossary_us.json'

 ---

 ## 6.3 레퍼런스에서 스타일 가이드 자동 추출

In [8]:
STYLE_EXTRACTION_PROMPT = """영어 번역본을 분석하여 스타일 가이드를 추출해주세요.

## 샘플
{samples}

반드시 JSON 형식으로 응답:
```json
{{
  "spelling_rules": ["규칙"],
  "tone": "톤 설명",
  "pronoun_usage": "인칭 패턴",
  "summary": "요약 (한국어)"
}}
```"""


In [9]:
def extract_style_guide(en_data: dict, locale: str) -> dict:
    samples = [f"- {s}" for s in list(en_data.values())[:15] if s]
    prompt = STYLE_EXTRACTION_PROMPT.format(samples="".join(samples))
    response = call_claude(prompt)

    try:
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        return {"locale": locale, "style_guide": json.loads(json_str)}
    except:
        return {"locale": locale, "style_guide": {}}


In [10]:
print("US 영어 스타일 가이드 추출 중...")
us_style_data = extract_style_guide(en_us_faq, "US")

print("\nUS 영어 스타일 가이드:")
for key, value in us_style_data["style_guide"].items():
    print(f"  {key}: {value}")

save_results(us_style_data, "style_guide_us.json")


US 영어 스타일 가이드 추출 중...

US 영어 스타일 가이드:
  spelling_rules: ['제품명은 정확한 공식 명칭 사용 (Samsung Cloud, Samsung Internet, Samsung Notes, Samsung Pass, Samsung Daily)', '기술 용어의 일관된 표기 (sync/backup, backup/restore)', "메뉴 경로는 '>' 기호로 구분 (Settings > Samsung Account > Samsung Cloud)", '괄호를 사용한 부가 설명 제공 (Events (Calendar), version 3.X or higher)', '하이픈을 사용한 목록 구조화', '질문 형태의 소제목에 물음표 사용']
  tone: 공식적이고 설명적인 톤. 기술 문서의 특성상 객관적이고 정확한 정보 전달에 중점을 둔 formal한 어조
  pronoun_usage: 2인칭 'you/your' 중심 사용으로 사용자와의 직접적 소통. 1인칭은 거의 사용하지 않으며, 사용자 관점에서 서술
  summary: Samsung Cloud 서비스에 대한 기술 문서로, 제품 기능 설명과 FAQ 형식을 결합한 사용자 가이드. 공식적이면서도 접근하기 쉬운 언어로 복잡한 기술적 내용을 체계적으로 정리하여 제시하는 스타일을 보임
✓ 저장 완료: lab_outputs/style_guide_us.json


'lab_outputs/style_guide_us.json'

 ---

 ## 6.4 평가 피드백 수집

In [11]:
def get_feedback(item_id: str) -> str:
    parts = []
    
    for e in faithfulness_data.get("glossary", []):
        if e["id"] == item_id and e.get("issues"):
            parts.append(f"[충실도] {', '.join(e['issues'])}")
    
    for e in terminology_data.get("glossary", []):
        if e["id"] == item_id and e.get("issues"):
            parts.append(f"[용어] {', '.join(e['issues'])}")
    
    for e in culture_tone_data.get("glossary", []):
        if e["id"] == item_id and e.get("issues"):
            parts.append(f"[톤] {', '.join(e['issues'])}")
    
    return "\n".join(parts) if parts else "특별한 문제점 없음"


 ---

 ## 6.5 US 영어 타겟팅 번역

In [12]:
TRANSLATION_PROMPT = """삼성 공식 번역가로서 한국어를 {locale} 영어로 번역해주세요.

## 원문
{source}

## 스타일 가이드
{style_guide}

## 용어집
{glossary}

## 이전 피드백
{feedback}

번역 결과만 출력하세요."""


In [13]:
def translate_targeted(source: str, item_id: str, glossary: dict, style: dict, locale: str) -> str:
    glossary_text = "\n".join([f"- {k} → {v}" for k, v in glossary.items()])
    style_text = style.get("summary", "")
    
    prompt = TRANSLATION_PROMPT.format(
        locale=locale,
        source=source,
        style_guide=style_text,
        glossary=glossary_text,
        feedback=get_feedback(item_id)
    )
    return call_claude(prompt).strip()


In [14]:
us_translations = []

print("US 영어 번역 실행 중...")
print("=" * 80)

for item in faq_items:
    translation = translate_targeted(
        item["ko"], item["id"],
        us_glossary_data["glossary"],
        us_style_data["style_guide"],
        "US"
    )
    
    us_translations.append({
        "id": item["id"],
        "source": item["ko"],
        "translation": translation,
        "reference": en_us_faq.get(item["id"], ""),
        "locale": "US"
    })
    
    print(f"\n[{item['id']}]")
    print(f"번역: {translation}")
    print(f"참조: {en_us_faq.get(item['id'], '')}")


US 영어 번역 실행 중...

[IDS_FAQ_SC_MAIN_HEADER_02]
번역: Do I need a Samsung account to use Samsung Cloud?
참조: Do I need a Samsung account to use Samsung Cloud?

[IDS_FAQ_SC_MAIN_HEADER_03]
번역: What happens if I remove my Samsung account from the device?
참조: What happens if I remove my Samsung account?

[IDS_FAQ_SC_ACCESSING_TEXT]
번역: After accessing the Samsung Cloud webpage, please contact us through Contact us using "1:1 inquiry." We can unblock access after verifying your user information.
참조: After accessing the Samsung Cloud web page, contact us through the "1:1 inquiry" in "Contact us". We will unlock your Samsung Cloud after confirming your user information.

[IDS_FAQ_SC_MAIN_HEADER_13]
번역: I don't currently have a Galaxy device. Is there a way to check my data or information that was stored and synced to Samsung services?
참조: I do not have a Galaxy device now. Is there a way to check my data or information stored and synchronized with Samsung services?

[IDS_FAQ_GO_DEVICE_NOTES_UL_01

 ---

 ## 6.6 US 번역 레퍼런스 유사도 평가

In [15]:
SIMILARITY_EVAL_PROMPT = """번역문과 레퍼런스 유사도를 평가해주세요.

원문: {source}
번역: {translation}
레퍼런스: {reference}

점수: 5=거의동일, 4=의미동일, 3=유사, 0-2=다름

JSON 형식으로 응답:
```json
{{"score": <0-5>, "differences": ["차이점"], "reasoning": "근거"}}
```"""

def eval_similarity(source: str, translation: str, reference: str) -> dict:
    prompt = SIMILARITY_EVAL_PROMPT.format(source=source, translation=translation, reference=reference)
    response = call_claude(prompt)
    
    try:
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        return json.loads(json_str)
    except:
        return {"score": 0, "differences": ["파싱실패"], "reasoning": ""}


In [16]:
print("US 번역 유사도 평가 중...")

us_similarity_evals = []
for item in us_translations:
 result = eval_similarity(item["source"], item["translation"], item["reference"])
 us_similarity_evals.append({"id": item["id"], **result})
 print(f"[{item['id']}] 유사도: {result['score']}/5")


US 번역 유사도 평가 중...
[IDS_FAQ_SC_MAIN_HEADER_02] 유사도: 5/5
[IDS_FAQ_SC_MAIN_HEADER_03] 유사도: 4/5
[IDS_FAQ_SC_ACCESSING_TEXT] 유사도: 4/5
[IDS_FAQ_SC_MAIN_HEADER_13] 유사도: 4/5
[IDS_FAQ_GO_DEVICE_NOTES_UL_01] 유사도: 3/5
[IDS_FAQ_GO_MAIN_HEADER_27] 유사도: 4/5


 ---

 ## 6.7 US 번역 품질 재평가 (충실도/용어/톤)

In [17]:
# 충실도 평가 프롬프트
FAITHFULNESS_PROMPT = """원문과 번역의 충실도를 평가해주세요.

원문: {source}
번역: {translation}

점수: 5=완벽, 4=경미차이, 3=일부누락, 0-2=오류

JSON 형식:
```json
{{"score": <0-5>, "issues": ["문제점"], "reasoning": "근거"}}
```"""

# 용어 평가 프롬프트
TERMINOLOGY_PROMPT = """용어집 준수 여부를 평가해주세요.

번역: {translation}
용어집: {glossary}

점수: 5=완벽준수, 4=경미위반, 3=다수위반, 0-2=심각위반

JSON 형식:
```json
{{"score": <0-5>, "issues": ["문제점"], "reasoning": "근거"}}
```"""

# 톤 평가 프롬프트
TONE_PROMPT = """번역의 톤과 스타일을 평가해주세요.

번역: {translation}
스타일가이드: {style_guide}

점수: 5=완벽, 4=경미차이, 3=불일치, 0-2=부적절

JSON 형식:
```json
{{"score": <0-5>, "issues": ["문제점"], "reasoning": "근거"}}
```"""


In [18]:
def eval_faithfulness(source: str, translation: str) -> dict:
    prompt = FAITHFULNESS_PROMPT.format(source=source, translation=translation)
    response = call_claude(prompt)
    try:
        if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        return json.loads(json_str)
    except:
        return {"score": 0, "issues": ["파싱실패"], "reasoning": ""}

def eval_terminology(translation: str, glossary: dict) -> dict:
    glossary_text = "\n".join([f"- {k} → {v}" for k, v in glossary.items()])
    prompt = TERMINOLOGY_PROMPT.format(translation=translation, glossary=glossary_text)
    response = call_claude(prompt)
    try:
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        return json.loads(json_str)
    except:
        return {"score": 0, "issues": ["파싱실패"], "reasoning": ""}

def eval_tone(translation: str, style_guide: dict) -> dict:
    style_text = style_guide.get("summary", "")
    prompt = TONE_PROMPT.format(translation=translation, style_guide=style_text)
    response = call_claude(prompt)
    try:
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        return json.loads(json_str)
    except:
        return {"score": 0, "issues": ["파싱실패"], "reasoning": ""}


In [19]:
print("\nUS 번역 품질 재평가 중 (충실도/용어/톤)...")
print("=" * 80)

us_quality_evals = []

for item in us_translations:
    faith = eval_faithfulness(item["source"], item["translation"])
    term = eval_terminology(item["translation"], us_glossary_data["glossary"])
    tone = eval_tone(item["translation"], us_style_data["style_guide"])
    
    us_quality_evals.append({
        "id": item["id"],
        "faithfulness": faith,
        "terminology": term,
        "tone": tone
    })
    
    print(f"\n[{item['id']}]")
    print(f"  충실도: {faith['score']}/5")
    print(f"  용어: {term['score']}/5")
    print(f"  톤: {tone['score']}/5")



US 번역 품질 재평가 중 (충실도/용어/톤)...

[IDS_FAQ_SC_MAIN_HEADER_02]
  충실도: 5/5
  용어: 5/5
  톤: 5/5

[IDS_FAQ_SC_MAIN_HEADER_03]
  충실도: 5/5
  용어: 5/5
  톤: 2/5

[IDS_FAQ_SC_ACCESSING_TEXT]
  충실도: 4/5
  용어: 5/5
  톤: 2/5

[IDS_FAQ_SC_MAIN_HEADER_13]
  충실도: 5/5
  용어: 5/5
  톤: 2/5

[IDS_FAQ_GO_DEVICE_NOTES_UL_01]
  충실도: 4/5
  용어: 5/5
  톤: 4/5

[IDS_FAQ_GO_MAIN_HEADER_27]
  충실도: 4/5
  용어: 2/5
  톤: 4/5


 ---

 ## 6.8 US 번역 결과 요약

In [20]:
# 점수 계산
similarity_avg = sum(e["score"] for e in us_similarity_evals) / len(us_similarity_evals)
faith_avg = sum(e["faithfulness"]["score"] for e in us_quality_evals) / len(us_quality_evals)
term_avg = sum(e["terminology"]["score"] for e in us_quality_evals) / len(us_quality_evals)
tone_avg = sum(e["tone"]["score"] for e in us_quality_evals) / len(us_quality_evals)

print("=" * 80)
print("US 영어 번역 결과 요약")
print("=" * 80)
print(f"\n레퍼런스 유사도: {similarity_avg:.2f}/5")
print(f"충실도: {faith_avg:.2f}/5")
print(f"용어 일관성: {term_avg:.2f}/5")
print(f"톤/스타일: {tone_avg:.2f}/5")
print("=" * 80)


US 영어 번역 결과 요약

레퍼런스 유사도: 4.00/5
충실도: 4.50/5
용어 일관성: 4.50/5
톤/스타일: 3.17/5


In [21]:
# 결과 저장
save_results({
    "translations": us_translations,
    "similarity_evals": us_similarity_evals,
    "quality_evals": us_quality_evals,
    "summary": {
        "similarity_avg": similarity_avg,
        "faithfulness_avg": faith_avg,
        "terminology_avg": term_avg,
        "tone_avg": tone_avg
    },
    "glossary": us_glossary_data["glossary"],
    "style_guide": us_style_data["style_guide"]
}, "translations_us_targeted.json")



✓ 저장 완료: lab_outputs/translations_us_targeted.json


'lab_outputs/translations_us_targeted.json'

 ---

 ## 6.9 🎯 미션: GB 영어 타겟팅 번역



 US 영어와 동일한 방식으로 GB 영어 번역을 구현해보세요!

 레퍼런스(en-rGB_faq.json)에서 용어집과 스타일 가이드를 추출합니다.

 ### 미션 1: GB 영어 용어집 자동 추출



 힌트: `extract_glossary(ko_faq, en_gb_faq, "GB")` 사용

In [22]:
# TODO: 아래 코드를 완성하세요
print("GB 영어 용어집 추출 중...")

# gb_glossary_data = extract_glossary(???, ???, ???)

# print("\nGB 영어 용어집:")
# for ko, en in gb_glossary_data["glossary"].items():
#     print(f"  {ko} → {en}")

# save_results(gb_glossary_data, "glossary_gb.json")


GB 영어 용어집 추출 중...


 ### 미션 2: GB 영어 스타일 가이드 자동 추출



 힌트: `extract_style_guide(en_gb_faq, "GB")` 사용

In [23]:
# TODO: 아래 코드를 완성하세요
print("GB 영어 스타일 가이드 추출 중...")

# gb_style_data = extract_style_guide(???, ???)

# print("\nGB 영어 스타일 가이드:")
# for key, value in gb_style_data["style_guide"].items():
#     print(f"  {key}: {value}")

# save_results(gb_style_data, "style_guide_gb.json")


GB 영어 스타일 가이드 추출 중...


 ### 미션 3: US vs GB 용어집 차이 비교



 추출된 US와 GB 용어집을 비교하여 차이점을 확인하세요.

In [24]:
# TODO: 아래 코드를 완성하세요
print("=" * 80)
print("US vs GB 용어집 차이")
print("=" * 80)

# us_glossary = us_glossary_data["glossary"]
# gb_glossary = gb_glossary_data["glossary"]

# all_keys = set(us_glossary.keys()) | set(gb_glossary.keys())
# for ko in all_keys:
#     us_term = us_glossary.get(ko, "-")
#     gb_term = gb_glossary.get(ko, "-")
#     if us_term != gb_term:
#         print(f"  {ko}: US={us_term}, GB={gb_term}")


US vs GB 용어집 차이


 ### 미션 4: GB 영어 번역 실행



 힌트: `translate_targeted()` 함수에 GB 용어집과 스타일 가이드 전달

In [25]:
# TODO: 아래 코드를 완성하세요
gb_translations = []

print("GB 영어 번역 실행 중...")
print("=" * 80)

# for item in faq_items:
#     translation = translate_targeted(
#         item["ko"], item["id"],
#         ???,  # GB 용어집
#         ???,  # GB 스타일 가이드
#         "GB"
#     )
#     
#     gb_translations.append({
#         "id": item["id"],
#         "source": item["ko"],
#         "translation": translation,
#         "reference": en_gb_faq.get(item["id"], ""),
#         "locale": "GB"
#     })
#     
#     print(f"\n[{item['id']}]")
#     print(f"번역: {translation}")
#     print(f"참조: {en_gb_faq.get(item['id'], '')}")


GB 영어 번역 실행 중...


 ### 미션 5: GB 번역 유사도 평가



 힌트: `eval_similarity()` 함수 사용

In [26]:
# TODO: 아래 코드를 완성하세요
print("\nGB 번역 유사도 평가 중...")

gb_similarity_evals = []

# for item in gb_translations:
#     result = eval_similarity(item["source"], item["translation"], item["reference"])
#     gb_similarity_evals.append({"id": item["id"], **result})
#     print(f"[{item['id']}] 유사도: {result['score']}/5")



GB 번역 유사도 평가 중...


 ### 미션 6: GB 번역 품질 재평가 (충실도/용어/톤)



 힌트: `eval_faithfulness()`, `eval_terminology()`, `eval_tone()` 함수 사용

In [27]:
# TODO: 아래 코드를 완성하세요
print("\nGB 번역 품질 재평가 중 (충실도/용어/톤)...")
print("=" * 80)

gb_quality_evals = []

# for item in gb_translations:
#     faith = eval_faithfulness(item["source"], item["translation"])
#     term = eval_terminology(item["translation"], gb_glossary_data["glossary"])
#     tone = eval_tone(item["translation"], gb_style_data["style_guide"])
#     
#     gb_quality_evals.append({
#         "id": item["id"],
#         "faithfulness": faith,
#         "terminology": term,
#         "tone": tone
#     })
#     
#     print(f"\n[{item['id']}]")
#     print(f"  충실도: {faith['score']}/5")
#     print(f"  용어: {term['score']}/5")
#     print(f"  톤: {tone['score']}/5")



GB 번역 품질 재평가 중 (충실도/용어/톤)...


 ### 미션 7: GB 번역 결과 요약 및 저장

In [28]:
# TODO: 아래 코드를 완성하세요

# GB 점수 계산
# gb_similarity_avg = sum(e["score"] for e in gb_similarity_evals) / len(gb_similarity_evals)
# gb_faith_avg = sum(e["faithfulness"]["score"] for e in gb_quality_evals) / len(gb_quality_evals)
# gb_term_avg = sum(e["terminology"]["score"] for e in gb_quality_evals) / len(gb_quality_evals)
# gb_tone_avg = sum(e["tone"]["score"] for e in gb_quality_evals) / len(gb_quality_evals)

# print("=" * 80)
# print("GB 영어 번역 결과 요약")
# print("=" * 80)
# print(f"\n레퍼런스 유사도: {gb_similarity_avg:.2f}/5")
# print(f"충실도: {gb_faith_avg:.2f}/5")
# print(f"용어 일관성: {gb_term_avg:.2f}/5")
# print(f"톤/스타일: {gb_tone_avg:.2f}/5")
# print("=" * 80)

# GB 결과 저장
# save_results({
#     "translations": gb_translations,
#     "similarity_evals": gb_similarity_evals,
#     "quality_evals": gb_quality_evals,
#     "summary": {
#         "similarity_avg": gb_similarity_avg,
#         "faithfulness_avg": gb_faith_avg,
#         "terminology_avg": gb_term_avg,
#         "tone_avg": gb_tone_avg
#     },
#     "glossary": gb_glossary_data["glossary"],
#     "style_guide": gb_style_data["style_guide"]
# }, "translations_gb_targeted.json")


 ---

 ## 6.10 US vs GB 최종 비교



 미션을 완료한 후 아래 코드를 실행하여 US와 GB 결과를 비교하세요.

In [ ]:
# 미션 완료 후 주석 해제하여 실행
# print("=" * 100)
# print("US vs GB 타겟팅 번역 결과 비교")
# print("=" * 100)

# print(f"\n{'평가 항목':<20} {'US':<15} {'GB':<15}")
# print("-" * 50)
# print(f"{'레퍼런스 유사도':<20} {similarity_avg:<15.2f} {gb_similarity_avg:<15.2f}")
# print(f"{'충실도':<20} {faith_avg:<15.2f} {gb_faith_avg:<15.2f}")
# print(f"{'용어 일관성':<20} {term_avg:<15.2f} {gb_term_avg:<15.2f}")
# print(f"{'톤/스타일':<20} {tone_avg:<15.2f} {gb_tone_avg:<15.2f}")
# print("-" * 50)

# us_total = (similarity_avg + faith_avg + term_avg + tone_avg) / 4
# gb_total = (gb_similarity_avg + gb_faith_avg + gb_term_avg + gb_tone_avg) / 4
# print(f"{'종합 평균':<20} {us_total:<15.2f} {gb_total:<15.2f}")
# print("=" * 100)


 ---

 ## 6.11 Lab 6 완료

In [29]:
print("=" * 80)
print("Lab 6: 피드백 기반 재번역 완료")
print("=" * 80)

print("\n[핵심 학습 내용]")
print("  - 레퍼런스 데이터에서 용어집/스타일 가이드 자동 추출")
print("  - 추출된 리소스를 활용한 지역별 타겟팅 번역")
print("  - 재번역 후 충실도/용어/톤 재평가로 품질 검증")

print("\n[US 영어 결과]")
print(f"  - 레퍼런스 유사도: {similarity_avg:.2f}/5")
print(f"  - 충실도: {faith_avg:.2f}/5")
print(f"  - 용어 일관성: {term_avg:.2f}/5")
print(f"  - 톤/스타일: {tone_avg:.2f}/5")

print("\n[GB 영어 미션]")
print("  - 위 미션들을 완료하여 GB 영어 결과를 확인하세요!")

print("\n[저장된 파일]")
print(f"  - {OUTPUT_DIR}/glossary_us.json")
print(f"  - {OUTPUT_DIR}/style_guide_us.json")
print(f"  - {OUTPUT_DIR}/translations_us_targeted.json")

print("\n[다음 단계]")
print("  → Lab 7: 종합 분석")

print("\n" + "=" * 80)

Lab 6: 피드백 기반 재번역 완료

[핵심 학습 내용]
  - 레퍼런스 데이터에서 용어집/스타일 가이드 자동 추출
  - 추출된 리소스를 활용한 지역별 타겟팅 번역
  - 재번역 후 충실도/용어/톤 재평가로 품질 검증

[US 영어 결과]
  - 레퍼런스 유사도: 4.00/5
  - 충실도: 4.50/5
  - 용어 일관성: 4.50/5
  - 톤/스타일: 3.17/5

[GB 영어 미션]
  - 위 미션들을 완료하여 GB 영어 결과를 확인하세요!

[저장된 파일]
  - lab_outputs/glossary_us.json
  - lab_outputs/style_guide_us.json
  - lab_outputs/translations_us_targeted.json

[다음 단계]
  → Lab 7: 종합 분석

